&nbsp;

# BONUS - Extraction d'un état réduit par sélection
---

&nbsp;

> #### Pourquoi extraire un état réduit de cette manière ?

Quand on construit notre état réduit à partir de la PCA/EOF, on construit des combinaisons linéaires des variables originales. Les modes (EOF1, EOF2, etc.) n'existent pas physiquement et permettent surtout de maximiser la variance expliquée.

Le défaut, c'est que dans un projet où notre objectif est de capturer la dynamique thermique d'une zone géographique définie, entraîner notre modèle sur des données synthétiques peut devenir un obstacle à l'interprétabiltié causale et physique.

&nbsp;

Dans l'optique de pouvoir comparer avec notre état réduit initial, nous allons réaliser une sélection de *features* (une seconde famille de méthode pour réduire un jeu de donnée). Ces méthodes permettent de conserver les variables à variance élevée (celle qui sont les plus explicatives) ou à variance conditionnelle.

&nbsp;

---

&nbsp;

## Implémentation d'une méthode de sélection

Nous allons implémenter une combinaison simple (SINDy + Lasso) pour réduire notre jeu de données d'anomalies désaisonnalisées par la sélection.

On commence par charger notre jeu de données post analyse statistique et importons les librairies qui nous serons utiles. On *reshape* le champ de SST en une matrice 2D en "empilant" les dimensions spatiales (ainsi chaque localisation correspond à un élément de la colonne spatiale).

Le champ SST est initialement un champ spatio-temporel : 

$$
\text{SST}(t,\phi,\lambda)
$$

Et comme dans notre précédente extraction, nous le reformulons en une matrice de données :

$$
X \in \mathbb{R}^{T\times N}
$$

où :
- T = nombre d'instants temporels
- N = nombre de point spatiaux

Chaque colonne correspondra ainsi à une variable physique réelle : 

$$
X(t) = [x_1(t), x_2(t), \dots, x_N(t)]
$$

In [2]:
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso

ds = xr.open_dataset("data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")

sst = ds["analysed_sst"]

# Shaping from (time, lat, lon) to (time, space)
sstStacked = sst.stack(space=("latitude", "longitude"))
sstStacked = sstStacked.dropna("space")

X = sstStacked.values  # (T, Nspace)
dt = 1.0  # time step in days

En suite, on applique une standardisation colonne par colonne. C'est indispensable car :

- PCA/EOF maximise la variance :
$$
\text{max Var}(u^{\perp}X)
$$
$\to$ sans normalisation, les régions à forte variance dominent artificiellement.
- Lasso résout (voir la partie théorie de la méthode utilisée):
$$
\underset{\beta}{\text{min}}||y - X\beta||²_2 + \alpha||\beta||_1
$$
$\to$ cela correspond à une régression linéaire classique, plus un facteur dit "pénalisant" $\mathcal{l}_1$ qui dépend directement de l'échelle.

Par une normalisation du type :

$$
\tilde{x}(t) = \frac{x(t) - \mu}{\sigma}
$$

$\to$ on garantie que la sélection repose sur la dynamique plutôt que sur l'amplitude brute.

In [3]:
scaler = StandardScaler()
XScaled = scaler.fit_transform(X)

> #### Pourquoi réutilise-t-on la PCA ?

C'est le *twist*. On applique la PCA non pas pour utiliser les modes résultant comme variables d'état de notre état réduit mais plutôt comme outil intermédiaire. Elles nous servent uniquement commme élément d'observation/comparaison pour, par la suite, identifier les variables physiques du jeu qui structurent la dynamique globale (puis à les sélectionner).

In [4]:
nComponents = 100
pca = PCA(n_components=nComponents)

PCs = pca.fit_transform(XScaled)       # (T, K)
EOFs = pca.components_.T               # (Nspace, K)
explainedVar = pca.explained_variance_ratio_

Dans cette étape, nous classons les variables physiques en fonction de leur contribution aux modes de variabilité dominants.

In [7]:
topKModes = 30
importance = np.zeros(EOFs.shape[0])

for k in range(topKModes):
    importance += explainedVar[k] * np.abs(EOFs[:, k])

nCandidates = 20
candidateIndices = np.argsort(importance)[-nCandidates:]

Les différences finies sont utilisées comme approximation du premier ordre ; la sensibilité au bruit est prise en compte.

In [8]:
dXdt = np.gradient(XScaled, dt, axis=0)

In [9]:
target = PCs[:, 0]
dTargetdt = np.gradient(target, dt)

XCandidates = XScaled[:, candidateIndices]

lasso = Lasso(alpha=0.005, max_iter=10000)
lasso.fit(XCandidates, dTargetdt)

selectedMask = np.abs(lasso.coef_) > 1e-6
selectedIndices = candidateIndices[selectedMask]

In [10]:
XReducedScaled = XScaled[:, selectedIndices]
dXdtReduced = np.gradient(XReducedScaled, dt, axis=0)

In [12]:
dsReduced = xr.Dataset(
    {
        "XReduced": (("time", "space"), XReducedScaled),
        "dXdtReduced": (("time", "space"), dXdtReduced),
    },
    coords={
        "time": ds["time"],
        "space": selectedIndices,
    },
    attrs={
        "description": "Reduced physical state obtained via EOF-based ranking and sparse dynamic selection"
    }
)

dsReduced.to_netcdf("data/processed/sstReducedStateCOPERNICUS20102019.nc")


In [1]:
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso

# Load deseasonalized SST data
ds = xr.open_dataset("data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")

# Extract SST data and adapt its shape
sst = ds["analysed_sst"]
sstStacked = sst.stack(space=("latitude", "longitude"))
sstStacked = sstStacked.dropna("space")

X = sstStacked.values  # shape (Nt, Nspace)

dt = 1.0  # time step in days

scaler = StandardScaler()
XScaled = scaler.fit_transform(X)

nComponents = 100
pca = PCA(n_components=nComponents)

PCs = pca.fit_transform(XScaled)     # (T, K)
EOFs = pca.components_.T               # (N, K)
explainedVar = pca.explained_variance_ratio_

topKModes = 30

importance = np.zeros(EOFs.shape[0])

for k in range(topKModes):
    importance += explainedVar[k] * np.abs(EOFs[:, k])

nCandidates = 20
candidateIndices = np.argsort(importance)[-nCandidates:]

dXdt = np.gradient(XScaled, dt, axis=0)

# Sparse regression using Lasso

targetIndex = 0  # variable physique ou composante d’intérêt

XCandidates = XScaled[:, candidateIndices]
yTarget = dXdt[:, targetIndex]

lasso = Lasso(alpha=0.005)
lasso.fit(XCandidates, yTarget)

selectedMask = np.abs(lasso.coef_) > 1e-6
selectedIndices = candidateIndices[selectedMask]

XReduced = X[:, selectedIndices]
dXdtReduced = np.gradient(XReduced, dt, axis=0)

# Save the reduced dataset
dsReduced = xr.Dataset(
    {
        "XReduced": (("time", "space"), XReduced),
        "dXdtReduced": (("time", "space"), dXdtReduced),
    },
    coords={
        "time": ds["time"],
        "space": selectedIndices,
    },
)

dsReduced.to_netcdf("data/processed/sstReducedState2COPERNICUS20102019.nc")